In [ ]:
#| default_exp oas_to_fc

In [ ]:
#| export
from llmcam.oas_to_requests import *
from llmcam.fn_to_fc import *

In [ ]:
#| export
import os
import json
import jsonref
import requests
from pprint import pp
import openai

In [ ]:
url = 'https://petstore3.swagger.io/api/v3/openapi.json'
filename = 'oas.json'
response = requests.get(url)
with open(filename, 'wb') as file:
    file.write(response.content)

In [ ]:
with open('oas.json', 'r') as f:
    oas = jsonref.loads(f.read()) # it's important to load with jsonref, as explained below
for path, methods in oas["paths"].items():
    for method, spec_with_ref in methods.items():
        print(path, ":", method, ":", spec_with_ref['operationId'], ":", spec_with_ref['summary'])

/pet : put : updatePet : Update an existing pet
/pet : post : addPet : Add a new pet to the store
/pet/findByStatus : get : findPetsByStatus : Finds Pets by status
/pet/findByTags : get : findPetsByTags : Finds Pets by tags
/pet/{petId} : get : getPetById : Find pet by ID
/pet/{petId} : post : updatePetWithForm : Updates a pet in the store with form data
/pet/{petId} : delete : deletePet : Deletes a pet
/pet/{petId}/uploadImage : post : uploadFile : uploads an image
/store/inventory : get : getInventory : Returns pet inventories by status
/store/order : post : placeOrder : Place an order for a pet
/store/order/{orderId} : get : getOrderById : Find purchase order by ID
/store/order/{orderId} : delete : deleteOrder : Delete purchase order by ID
/user : post : createUser : Create user
/user/createWithList : post : createUsersWithListInput : Creates list of users with given input array
/user/login : get : loginUser : Logs user into the system
/user/logout : get : logoutUser : Logs out curr

In [ ]:
#| export
def oas_to_fc(openapi_spec):
    functions = []

    for path, methods in openapi_spec["paths"].items():
        for method, spec_with_ref in methods.items():
            # 1. Resolve JSON references.
            spec = jsonref.replace_refs(spec_with_ref)

            # 2. Extract a name for the functions.
            function_name = spec.get("operationId")

            # 3. Extract a description and parameters.
            desc = spec.get("description") or spec.get("summary", "")

            schema = {"type": "object", "properties": {}}

            req_body = (
                spec.get("requestBody", {})
                .get("content", {})
                .get("application/json", {})
                .get("schema")
            )
            if req_body:
                schema["properties"]["requestBody"] = req_body

            params = spec.get("parameters", [])
            if params:
                param_properties = {
                    param["name"]: param["schema"]
                    for param in params
                    if "schema" in param
                }
                schema["properties"]["parameters"] = {
                    "type": "object",
                    "properties": param_properties,
                }

            functions.append(
                {"type": "function", "function": {"name": function_name, "description": desc, "parameters": schema}}
            )

    return functions

In [ ]:
#| eval: false
tools = oas_to_fc(oas)
messages = [{"role":"system", "content":"You are a helpful system administrator. Use the supplied tools to assist the user."}]
complete(messages, "user", "Hi, how many pets are sold?", tools=tools, aux_fn=generate_request_by_operation_id)

https://petstore3.swagger.io/api/v3/store/inventory None


('assistant',
 "It seems that I couldn't retrieve the inventory information at the moment. Please try again later or check if there is any issue with the system.")